# RL 训练

In [1]:
import os
# fix：https://github.com/unslothai/unsloth/issues/2299#issuecomment-2782067709
os.environ["VLLM_USE_V1"] = '0'
# 国内需要禁止统计，否则会卡在模型加载的地方（连不到外网）
os.environ["UNSLOTH_DISABLE_STATISTICS"] = "0"

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/data/countdown/output/models/Qwen2.5-1.5B-Instruct", # change to your model path
    # 使用 merge.ipynb 合并后的模型作为 RL 的基座，从而解决冷启动的问题。
    # model_name = "/home/cynic/models/qwen2.5-1.5b-sft-distill-merged",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    local_files_only=True,
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank * 2,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-20 14:51:08 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /home/cynic/models/Qwen2.5-1.5B-Instruct with actual GPU utilization = 46.74%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 23.99 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-20 14:51:21 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-20 14:51:21 [model_runner.py:1146] Model loading took 1.2131 GiB and 0.873978 seconds
INFO 04-20 14:51:22 [worker.py:267] Memory profiling takes 0.88 seconds
INFO 04-20 14:51:22 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.99GiB) x gpu_memory_utilization (0.47) = 11.21GiB
INFO 04-20 14:51:22 [worker.py:267] model weights take 1.21GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.23GiB; the rest of the memory reserved for KV Cache is 8.69GiB.
INFO 04-20 14:51:22 [executor_base.py:112] # cuda blocks: 20347, # CPU blocks: 14043
INFO 04-20 14:51:22 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 158.96x
INFO 04-20 14:51:23 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. 

Capturing CUDA graph shapes:   0%|          | 0/31 [00:00<?, ?it/s]

INFO 04-20 14:51:42 [model_runner.py:1598] Graph capturing finished in 18 secs, took 0.54 GiB
INFO 04-20 14:51:42 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 20.81 seconds


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [2]:
from constant import SYSTEM_PROMPT, USER_PROMPT_TPL, parse_user_prompt
from datasets import load_dataset
def get_countdown_questions():
    data = load_dataset("json", data_files="data/rl_data_simple.jsonl")["train"]
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', "content": parse_user_prompt(USER_PROMPT_TPL, x["numbers"], x["target"])}
        ],
    })
    return data

dataset = get_countdown_questions()

dataset[0]

{'numbers': [77, 7, 77],
 'target': 7,
 'ground_truth_solution': '(77 / 77) * 7',
 'prompt': [{'content': 'You are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer.',
   'role': 'system'},
  {'content': 'Using the numbers 77, 7, 77, create an equation that equals 7. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once, and you must use all the numbers. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer>(1 + 2) / 3</answer>. Think step by step inside <think> tags.',
   'role': 'user'}]}

In [3]:
import re

In [4]:
def extract_xml_answer(response):
    pattern = r"<answer>(.*?)</answer>"
    match = re.search(pattern, response, re.DOTALL)
    return match.group(1) if match else ""

# Reward function that checks if the completion follows a more relaxed format
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<think>.*?</think>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r, re.DOTALL) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

# 正确性奖励
def correctness_reward_func(completions, numbers, target, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]

    def correct(numbers, target, solution):
        try:
            if '=' in solution:
                # 删除掉等号以及等号后的内容
                solution = solution.split('=')[0]
            if not eval(solution) == target:
                # 如果 solution 计算结果不等于 target，则认为不正确
                return False, "Solution is not equal to target"
            # Define a regex pattern that only allows numbers, operators, parentheses, and whitespace
            allowed_pattern = r'^[\d+\-*/().\s]+$'
            if not re.match(allowed_pattern, solution):
                # 如果 solution 不符合要求，则认为不正确
                return False, "Solution is not allowed"
            # 检查 solution 是否使用了所有数字
            used_numbers = [int(n) for n in re.findall(r'\d+', solution)]
            if sorted(used_numbers) != sorted(numbers):
                # 如果 solution 没有使用所有数字没有仅使用一次，则认为不正确
                return False, "Solution is not using all numbers or each number is not used only once"
            return True, "Solution is correct"
        except Exception as e:
            # 计算失败，则认为不正确
            return False, f"Solution calculation failed: {e}"
    
    scores = []
    for i, r in enumerate(extracted_responses):
        scores.append(2 if correct(numbers[i], target[i], r)[0] else 0)
    
    return scores

In [5]:
# test reward function
from datasets import load_from_disk
eval_dataset = load_from_disk("output/test_vllm__qwen2.5-1.5b-sft-distill-lora_results")
idx = 11
eval_dataset[idx]

{'target': 656,
 'numbers': [85, 36, 16, 519],
 'prompt': 'Using the numbers 85, 36, 16, 519, create an equation that equals 656. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once, and you must use all the numbers. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer>(1 + 2) / 3</answer>. Think step by step inside <think> tags.',
 'completion': '<think>Okay, let\'s see. I need to make an equation using 85, 36, 16, and 519, each exactly once, with basic operations, and the result should be 656. Hmm. Let me start by thinking about how these numbers can combine.\n\nFirst, maybe look at the larger numbers like 519 and 85. If I subtract them, 519 - 85 is 434. That\'s not close to 656. Adding them would get way too big. So subtraction might not help here. Maybe multiplication? Let\'s try multiplying some of the smaller numbers together. 36 * 16 is 576. Then adding 

In [6]:
print(f'Soft format reward: {soft_format_reward_func([[{"content": eval_dataset[idx]["completion"]}]])}')
score = correctness_reward_func(
    [[{"content": eval_dataset[idx]["completion"]}]],
    [eval_dataset[idx]["numbers"]],
    [eval_dataset[idx]["target"]])
print(f'Correctness reward: {score}')

Soft format reward: [0.5]
Correctness reward: [2]


In [7]:
import wandb
wandb.init(project="countdown-rl-simple")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: swulling to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.01,
    lr_scheduler_type = "constant",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 32,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    vllm_max_model_len= 2048,
    max_prompt_length = 1024,
    max_completion_length = 1024,
    temperature = 1.0, # set to 1.0 for more diverse responses
    num_train_epochs = 5, # Set to 1 for a full training run
    #max_steps = 250,
    save_steps = 100,
    max_grad_norm = 0.1,
    output_dir = "output/rl2",
    beta=0.001,
    report_to = "wandb", # Can use Weights & Biases
)

In [9]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        soft_format_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 5 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 73,859,072/5,000,000,000 (1.48% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / soft_format_reward_func,rewards / correctness_reward_func
1,-0.000000,0.171875,0.296149,262.312500,0.000000,0.109375,0.062500
2,0.000000,0.109375,0.217785,294.812500,0.000916,0.109375,0.000000
3,0.000000,0.125000,0.261980,275.437500,0.000816,0.062500,0.062500
4,0.000000,0.296875,0.416193,314.562500,0.000851,0.109375,0.187500
5,0.000000,0.218750,0.512684,290.906250,0.001052,0.093750,0.125000
6,0.000000,0.328125,0.554279,293.093750,0.001216,0.203125,0.125000
7,0.000000,0.156250,0.187251,322.843750,0.003098,0.156250,0.000000
8,0.000000,0.187500,0.242518,253.562500,0.002419,0.187500,0.000000
9,0.000000,0.218750,0.242518,310.156250,0.002461,0.218750,0.000000
10,0.000000,0.234375,0.372012,280.156250,0.004520,0.171875,0.062500


Unsloth: Will smartly offload gradients to save VRAM!


<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?


TrainOutput(global_step=1250, training_loss=0.0021190986134886707, metrics={'train_runtime': 12606.1712, 'train_samples_per_second': 0.397, 'train_steps_per_second': 0.099, 'total_flos': 0.0, 'train_loss': 0.0021190986134886707})

https://wandb.ai/swulling/countdown-rl-simple/runs/w9yvf1gu?nw=nwuserswulling

In [10]:
model.save_pretrained("output/qwen2.5-1.5b-rl-v2-lora")  # Local saving lora weights
tokenizer.save_pretrained("output/qwen2.5-1.5b-rl-v2-lora")

('output/qwen2.5-1.5b-rl-v2-lora/tokenizer_config.json',
 'output/qwen2.5-1.5b-rl-v2-lora/special_tokens_map.json',
 'output/qwen2.5-1.5b-rl-v2-lora/vocab.json',
 'output/qwen2.5-1.5b-rl-v2-lora/merges.txt',
 'output/qwen2.5-1.5b-rl-v2-lora/added_tokens.json',
 'output/qwen2.5-1.5b-rl-v2-lora/tokenizer.json')

vllm inference
```bash
vllm serve Qwen/Qwen2.5-1.5B-Instruct --port 8100 --api-key NLUKKXIJDZ91rpg1z --enforce-eager  --max-model-len 4096 --enable-lora --max-lora-rank 64 --lora-modules qwen2.5-1.5b-rl-v2-lora=output/qwen2.5-1.5b-rl-v2-lora

CURATOR_VIEWER=1 python eval.py --provider vllm --data_path data/test_simple.jsonl --model_name qwen2.5-1.5b-rl-v2-lora --temperature 0.01 --max_tokens 2048

https://curator.bespokelabs.ai/datasets/fc75d8c833ba4d7984d925604337a9d5  

Accuracy: 45/100 (45.00%)
```

模型分析：

1. 模型学会了排列组合：`<think> 28 + 43 = 71 71 + 3 = 74 (too low) Let's try another combination 3 * 43 = 129 129 + 28 = 157 (works) </think> <answer>(3 * 43) + 28</answer>`
2. 因为有些问题拿不到正确性奖励，论文 [Understanding R1-Zero-Like Training: A Critical Perspective](https://arxiv.org/pdf/2503.20783) 中指出 GRPO 默认算法是有length bias的，所以模型会倾向于给不会的问题直接结束输出，从而限制了模型的性能。例子：`<think> 680 - 17 = 663 663 + 13 = 676 (too much) Let's try another approach 13 * 17 = 221 221 + 680 = 901 (too much) But if we consider division: 680 / 13 = 52 52 * 17 = 844 (still too much) However, I might have missed something simple initially. let me check again carefully (17 - 13) * 680 = 4080 (way off) but let's see if there is any other combination I overlooked I think my initial thought was right on this time 13 + 17 = 30 30 * 680 = 20400 (too far from 520) So after rechecking, it seems no valid solution fits exactly as per given constraints </think> <answer></answer>`